In [ ]:
## 패키지/라이브러리 설치
# %pip install -Uq docx2txt langchain_community langchain-text-splitters langchain_chroma

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] <archive url/path> ...

no such option: --upgrage


In [ ]:
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1. 문서 내용 읽고, 분할 ===========================================================
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1500,
    chunk_overlap= 200,
    length_function= len,
    is_separator_regex= False,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장 =============================================
## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

## 2.3 벡터 데이터베이스에 저장
## [방법 1] in memory
# database = Chroma.from_documents(
#     documents= document_list,
#     embedding=embedding,
# )


In [ ]:

## [방법 2] 로컬에 저장
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law',
# )

# database


In [4]:
## 로컬에 저장된 임베딩 데이터 가져오기

database = Chroma(
    collection_name = 'chroma-law',
    persist_directory='./chroma',
     embedding_function=embedding,
)
database

In [6]:
# 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ================================
## 3.1 사용자 질문
# query = '전세사기 피해자 금융지원에 대해 설명해주세요.'
query = '전세사기 피해자로 인정받기 위한 조건은?'

## 3.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=3)

# 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ============================
## 4.1 프롬프트 작성
prompt = '''
[identity]
-당신은 전세사기 피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''
## 4.2 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)

## 4.3 LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model = 'gpt-4o',)

## 4.4 LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기 피해자로 인정받기 위해서는 다음의 요건을 모두 갖추어야 합니다. 다만, 경매 또는 공매 절차가 완료된 임차인의 경우에는 1호 및 3호의 요건은 제외됩니다:\n\n1. 「주택임대차보호법」 제3조에 따라 주택의 인도와 주민등록을 완료하고, 같은 법 제3조의2제2항에 따라 임대차계약증서상의 확정일자를 갖출 것. 이는 임차권등기를 마친 경우 또는 전세권이 설정된 경우도 포함합니다.\n\n2. 임차인의 임차보증금이 5억 원 이하일 것. 다만, 임차보증금의 상한액은 전세사기피해지원위원회가 시ㆍ도별 여건 및 피해자의 여건 등을 고려하여 2억 원의 범위에서 상향 조정할 수 있습니다.\n\n3. 임대인의 파산 또는 회생절차 개시, 임차주택의 경매 또는 공매절차의 개시, 임차인의 집행권원 확보 등에 해당하여 2인 이상의 임차인에게 임차보증금반환채권의 변제를 받지 못하는 피해가 발생하였거나 발생할 것이 예상될 것.\n\n4. 임대인 등이 임차보증금을 반환할 능력이 없는 자에 대한 임차주택의 양도 또는 임차보증금을 반환할 능력 없이 다수의 주택을 취득ㆍ임대하는 등 임차보증금 반환채무를 이행하지 않을 의도가 있었다고 의심할 만한 상당한 이유가 있을 것.\n\n다만, 임차인이 보증금 반환을 위한 보증 또는 보험에 가입하였거나, 임차인의 보증금이 최우선변제가 가능한 경우, 또는 임차인이 대항력이나 우선변제권 행사를 통해 보증금을 회수할 수 있는 경우 등은 피해자로 인정되는 것에서 제외됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 436, 'prompt_tokens': 3111, 'total_tokens': 3547, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'reject